The Turán number of $C_5^-$
==============================

This notebook contains calculations for the Turán number of $C_5^-$

As the blowup of $K_4^-$ contains $C_5^-$, we can additionally assume 
that we work in $K_4^-$-free structures. 

1. The first cell sets up the combinatorial theory of $C_5^-$ and 
$K_4^-$-free 3-graphs (called TGp). In addition, it sets up the 
combinatorial theory on the same 3-graphs with vertices colored 
from 3 possible colors (called CTGp)

2. The second cell performs the basic calculation of upper bounding
edges in the theory. It gives the 1/4 + 1/1000 (in fact 0.250728)
upper bound.

3. That upper bound is used in the following cell. It lower bounds
k221-2*k311 and k222/k111 at the optimum.

4. There is a quick sanity check below that, to check the coefficients
of k221, k311, f222, k222

5. The last cell works in the colored theory, and shows that the bad
edges have number smaller than the missing good edges. It uses the upper
bound about k222/k111.

In [1]:
###
### This cell is just to set up the theory.
### In practice it is not needed, as the calculations (multiplication table and generated structures)
### are already done and saved. But it is here for completeness (and for re-runs from scratch)
###

from sage.algebras.flag_algebras import *

# These are helper functions, to deal with classical exclusion (not just induced)
def check_containment(smalls, larges):
    """
    Helper function to check is any of the smalls appears in each of the larges.

    INPUT:
    smalls - list of flags, must be from a theory with edges relation
    larges - list of flags, also must be from a theory with edges relation

    OUTPUT:
    list of booleans, i-th element represents if i-th large flag is free from all smalls
    """
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in larges]
    res = []
    for ll in lis:
        good = True
        for ss in sis:
            for _ in ll.isomorphic_substructures_iterator(ss):
                good = False
                break
            if not good:
                break
        res.append(good)
    return res

def check_containment_cert(smalls, large):
    """
    Helper function to check is any of the smalls appears in the large, and if yes,
    returns an injection of the small

    INPUT:
    smalls - list of flags, must be from a theory with edges relation
    large - a flag, also must be from a theory with edges relation

    OUTPUT:
    empty list ([]) if all smalls is avoided, otherwise [small, mapping] telling how to
    inject small into large with mapping
    """
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = IncidenceStructure(large.size(), large.blocks()['edges'])
    for ii, ss in enumerate(sis):
        for xx in lis.isomorphic_substructures_iterator(ss):
            return [xx, smalls[ii]]
    return None

# This is some hack to create the theory for 3-graphs without C5- and K4-
# up to size 7. It is easier to make them as extensions of 6 sized structures
# so this code does that.

# Reset three graphs, so nothing is excluded
TG = ThreeGraphTheory
TG.exclude()

# C5 minus
C5m = TG(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])

# flags of size 5
fl5 = TG.generate_flags(5)

# boolean vector indicating each element in fl5 if it has C5m
gs = check_containment([C5m], fl5)

# k4 and k4m (the two induced structures with size 4 excluded)
k4 = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]])
k4m = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])

# set the excluded structures. k4, k4m and all in fl5 containing C5m
exls = [k4, k4m] + [xx for ii, xx in enumerate(fl5) if not gs[ii]]
TG.exclude(exls)

# check the list of flags with size 5 and 6
fl5 = TG.generate_flags(5)
fl6 = TG.generate_flags(6)

# quick 3-graph identifier code. This will be the identifier for
# the theory of C5- free 3-graphs (any identifier working for 3-graphs can work here)
def _identify_hypergraph(n, ftype_points, edges):
    g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
              format='vertices_and_edges')
    partt = [[ii] for ii in ftype_points] + \
            [[ii for ii in range(n) if ii not in ftype_points]] + \
            [list(range(n,n+len(edges)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

# generator code. It should really just return TG, but for size 7 that takes too long
# so this hack just returns TG for size up to 6, and for 7 it generates all flags
# with this extension technique
def _gen(n):
    if n<=4:
        for xx in TG.generate_flags(n):
            yield xx.blocks()
    elif n==5:
        for xx in fl5:
            yield xx.blocks()
    elif n==6:
        for xx in fl6:
            yield xx.blocks()
    elif n==7:
        import itertools
        from tqdm import tqdm
        fl7_m = [[] for ii in range(35+1)]
        subs = list(itertools.combinations(range(6), int(2)))
        for xx in tqdm(fl6):
            xb = xx.blocks()['edges']
            for ii in range(15+1):
                for pps in itertools.combinations(subs, int(ii)):
                    xbp = [[pp[0], pp[1], 6] for pp in pps] + xb
                    flxp = TG(7, edges=xbp)
                    en = len(xbp)
                    if flxp not in fl7_m[en]:
                        if check_containment(exls, [flxp])[0]:
                            fl7_m[en].append(flxp)
        fl7 = [yy for xx in fl7_m for yy in xx]
        for xx in fl7:
            yield xx.blocks()
    else:
        #for n>=8 just return an empty list, this will not be called so doesn't 
        #really matter
        return []

# Create the theory based on this generator and identifier
TGp = CombinatorialTheory("No5Cm", _gen, _identify_hypergraph, edges=3)

# for sanity check, print the number of structures with size 5, 6, 7
# should be 9 55 1127
print(len(TGp.generate_flags(5)), len(TGp.generate_flags(6)), len(TGp.generate_flags(7)))






# This is code to create colored theories
# This is a default code for all color partition
def _identifyCT(k, order_partition, n, ftype_points, **kwargs):
    is_graph = (k==2)
    color_number = sum(len(xx) for xx in order_partition)
    edges = kwargs["edges"]
    Cs = [[cx[0] for cx in kwargs["C{}".format(ii)]] for ii in range(color_number)]
    g_parts = [[ii] for ii in ftype_points] + \
              [[ii for ii in range(n) if ii not in ftype_points]]
    ppadd = 0 if is_graph else len(edges)
    g_verts = list(range(n+ppadd+color_number))
    g_parts.append(list(range(n, n+ppadd)))

    g_parts += [[n+ppadd+ii for ii in partition_j] for partition_j in order_partition]
    
    if is_graph:
        g_edges = list(edges)
        for ii in range(color_number):
            g_edges += [(xx, n+ii) for xx in Cs[ii]]
    else:
        g_edges = [(i+n,x) for i,b in enumerate(edges) for x in b]
        for ii in range(color_number):
            g_edges += [(xx, n+len(edges)+ii) for xx in Cs[ii]]
    g = Graph([g_verts, g_edges], format='vertices_and_edges')
    blocks = tuple(g.canonical_label(partition=g_parts).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

# This is also a default code for all color partition
def _generateCT(base_theory, k, order_partition, n):
    color_number = sum(len(xx) for xx in order_partition)
    BT = base_theory
    for xx in BT.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        
        for yy in itertools.product(range(color_number), repeat=int(n)):
            yy = list(yy)
            Cs = {"C{}".format(cc):[[ii] for ii, oo in enumerate(yy) if oo==cc] for cc in range(color_number)}
            iden = _identifyCT(k==2, order_partition, n, [], edges=edges, **Cs)
            if iden not in unique:
                unique.append(iden)
                Cs["edges"] = edges
                yield Cs

# To make the default codes work for this specific case:
# The generator:
# Colors the elements of TGp (3-graphs without C5- and K4-), works on 3-uniform structures
# and the colors 0, 1, 2 are interchangeable (otherwise it would say [[0], [1], [2]]
def generate_colored(n):
    return _generateCT(TGp, 3, [[0, 1, 2]], n)

# Same for the identifier. Colors are interchangeable.
def identify_colored(n, ftype_points, edges, C0, C1, C2):
    return _identifyCT(3, [[0, 1, 2]], n, ftype_points, edges=edges, C0=C0, C1=C1, C2=C2)

# CTGp is the colored variant of TGp (NoC5m)
CTGp = CombinatorialTheory("ColoredNo5Cm", generate_colored, identify_colored, edges=3, C0=1, C1=1, C2=1)

# sanity check, the number of flags with size 4, 5, 6
# should be 18 132 2840
print(len(CTGp.generate_flags(4)), len(CTGp.generate_flags(5)), len(CTGp.generate_flags(6)))

9 55 1127
18 132 2840


In [10]:
###
### This is where the actual calculation starts.
### This part just gives a standard upper bound on the number of edges
###

# degree equality 
p2f4 = TGp.generate_flags(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]

pointed_edge = TGp(3, edges=[[0, 1, 2]], ftype=[0])

standard_assums = [degree_difference, -degree_difference, pointed_edge-1/4]


#run the optimizer
max_edge = TGp.optimize_problem(TGp(3, edges=[[0, 1, 2]]), 7, maximize=True, positives=standard_assums)

#should be around  0.25072863788449695 < 1/4 + 1/1000
print("\n\n", max_edge, "\n\n")

Ftypes constructed in 64.57s
Block sizes done in 1.18s
Block sizes are [5, 74, 32, 388, 178, 83, 97, 34, 45, 54, 31, 28, -1127, -192]
Calculating product matrices for 12 ftypes and 1127 structures
Ftype on 5 points with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3], [1, 2, 4]] is complete: : 12it [00:03,  3.45it/s]
Table calculation done in 3.49s
Target and constraint calculation done in 0.22s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Result is 0.25072863788449695
Iter:  1 Ap: 4.78e-01 Pobj: -5.9027977e+01 Ad: 2.34e-01 Dobj:  2.5209567e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -1.3500045e+02 Ad: 7.59e-01 Dobj:  3.0296639e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -1.4671819e+02 Ad: 7.86e-01 Dobj:  2.2062351e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.4535501e+02 Ad: 9.34e-01 Dobj: -1.5262505e-01 
Iter:  5 Ap: 9.05e-01 Pobj: -1.4433981e+02 Ad: 8.98e-01 Dobj: -1.8951766e-01 
Iter:  6 Ap: 9.06e-01 Pobj: -1.8608619e+02 Ad: 4.51e-01 Dobj: -1.9250127e-01 
Iter:  7 Ap: 5.52e-01 

In [2]:
###
### This code block deals with the uncolored part of the calculation (minimizing k221-2*k311 and k222/k111)
###




import itertools
# base edges for f222 (0, 1, 2) is the center and (3, 4, 5) is the good edge
be = [[0, 1, 2], [3, 4, 5], [0, 1, 5], [0, 2, 4], [1, 2, 3]]
# additional edges for f222. These are the edges we can include while still not violating the no C5m condition.
ae = [[0, 4, 5], [1, 3, 5], [2, 3, 4]]

# f221 and f311 just a sum of two elements, so can write it explicitely. They are typed, so no automorphism consideration is needed
f221 = TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [0, 2, 4]]) + TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [0, 2, 4], [2, 3, 4]])
f311 = TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [1, 2, 4]]) + TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [1, 2, 4], [0, 3, 4]])

# f222, it is a sum of 8 flags, so simply use this iterator to loop through all subsets of the additional edges
f222 = TGp(6, ftype=[0, 1, 2], edges=be)
for ii in [1, 2, 3]:
    for xx in itertools.combinations(ae, int(ii)):
        f222 = f222 + TGp(6, ftype=[0, 1, 2], edges=be+list(xx))

# these are the projected values. Here the automorphism groups are taken care of.
k221 = f221.project()
k311 = f311.project()
k222 = f222.project()

#standard positivity constraints.

# edge density bounds, the edge <= 1/4 + 1/1000 is used here from the previous cell
edge_lower = TGp(3, edges=[[0, 1, 2]]) - 1/4
edge_upper = 1/4 + 1/1000 - TGp(3, edges=[[0, 1, 2]])

# alpha for some of the calculations
alpha = 1-(1/100)

# the list of positivity assumptions
uncolored_assums = [edge_lower, edge_upper, degree_difference, -degree_difference]

# This is for minimizing f222, which is a flag, so technically it is the quotient k222 / k111
min_f222 = TGp.optimize_problem(f222, 7, maximize=False, positives=uncolored_assums)

# This is for minimizing k221 - (2*alpha)*k311
# On the optimal construction k221 = 2*k311, so alpha guarantees this is slightly positive
min_k2m3 = TGp.optimize_problem(k221 - (2*alpha)*k311, 7, maximize=False, positives=uncolored_assums)

#they should be around 0.19495185630466152 and 0.0003212179234714776
#sometimes the print statements get tangled in the output of the CSDP. But the variables are set,
#they can be checked in a different cell
print("\n\n", min_f222, "\n\n", min_k2m3, "\n\n")

Ftypes constructed in 65.71s
Block sizes done in 0.08s
Block sizes are [5, 74, 32, 388, 178, 83, 97, 34, 45, 54, 31, 28, -1127, -204]
Calculating product matrices for 12 ftypes and 1127 structures
Ftype on 5 points with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3], [1, 2, 4]] is complete: : 12it [00:03,  3.57it/s]
Table calculation done in 3.37s
Target and constraint calculation done in 0.79s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Result is 0.19495185630466152
Iter:  1 Ap: 1.73e-01 Pobj: -8.3448809e+01 Ad: 6.51e-02 Dobj:  5.3454227e-01 
Iter:  2 Ap: 8.03e-01 Pobj: -4.2124663e+02 Ad: 2.77e-01 Dobj:  2.2698892e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -5.7083161e+02 Ad: 8.39e-01 Dobj:  5.0819956e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -5.8089785e+02 Ad: 8.78e-01 Dobj:  1.1975895e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -5.7886145e+02 Ad: 8.51e-01 Dobj:  6.3523286e-02 
Iter:  6 Ap: 1.00e+00 Pobj: -7.5226615e+02 Ad: 4.81e-01 Dobj:  5.8723223e-02 
Iter:  7 Ap: 2.40e-01 

In [ ]:
# change objective functions such that only bipartite is optimal.
# number of 1-1-1 edges
# difference between parts

# or just do the extended kernel thing from multiple constructions

In [4]:
# to check the coefficients. These sanity checks are easy to do in general
print("k221 is: ", k221, "\n\nk311 is: ", k311, "\n\nf222 is: ", f222, "\n\nk222 is: ", k222)

k221 is:  Flag Algebra Element over Rational Field
0    - Flag on 5 points, ftype from [] with edges=[]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 1, 3]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 1, 3], [0, 1, 4]]
1/30 - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4], [0, 1, 3]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3]]
2/15 - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4], [0, 1, 3], [0, 2, 4]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3], [1, 2, 4]] 

k311 is:  Flag Algebra Element over Rational Field
0    - Flag on 5 points, ftype from [] with edges=[]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 1, 3]]
0    

In [13]:
###
### This is the code that performs the calculations on the colored theory
###

loglevel = 1

#edge with correct colors
C = CTGp(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]])
#pointed edge with correct colors
Cp = CTGp(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], ftype=[0])

#edge with bad colors (since color-blind, this includes the bad edges looking the wrong way too)
B = CTGp(3, edges=[[0, 1, 2]], C0=[[0], [1]], C1=[[2]])

#pointed edge with bad colors
Bp = CTGp(3, edges=[[0, 1, 2]], C0=[[0], [2]], C1=[[1]], C2=[], ftype=[0])

#missing edge with good colors
M = CTGp(3, edges=[], C0=[[0]], C1=[[1]], C2=[[2]])

#positivity assumptions
#each point, good edges are more than bad edges (divided by two due to the wrong color order)
#edge density is larger than 0.194 (from previous calculation)
colored_assums = [Cp - Bp/2, C - 194/1000]

#an optimal construction
#optim = blowup_construction(CTGp, 6, 3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], symmetric=True, variable=True)
#ssoptim = set_sum(optim)
#der_optims = derivatives(ssoptim, [1/3, 1/3])

#bad is less than missing
res = CTGp.optimize_problem(B + (-1)*M, 6, maximize=True, positives=colored_assums, certificate=True)
#should be around 1.5177109841127238e-08
#print("\n\n", res[0], "\n\n")
#print("\n\n", res, "\n\n")

Ftypes constructed in 8.84s
Block sizes done in 6.60s
Block sizes are [18, 15, 14, 36, 31, 41, 13, 21, 36, 34, 33, 30, 12, 21, 18, 13, 24, 17, 28, 14, -2840, -26]
Calculating product matrices for 20 ftypes and 2840 structures
Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]], C0=[[0]], C1=[[1]], C2=[[2], [3]] is complete: : 20it [00:01, 13.57it/s]    
Table calculation done in 1.48s
Target and constraint calculation done in 0.83s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.81e-02 Pobj: -9.1859489e+00 Ad: 8.88e-02 Dobj:  3.1364900e+01 
Iter:  2 Ap: 2.95e-01 Pobj: -4.8528365e+01 Ad: 2.31e-01 Dobj:  1.0281105e+01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.4559573e+02 Ad: 4.12e-01 Dobj:  2.2104889e+01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.6316715e+02 Ad: 8.69e-01 Dobj:  2.5751194e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -1.7346888e+02 Ad: 9.34e-01 Dobj:  1.9625288e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.8511884e+02 Ad: 7.96e-01 Dobj:  5.2702450e-02 
Iter: 

In [12]:
0.n(), (380328/6994159).n(), (870366/6994159).n(), (615342/6994159).n(), (810264/6994159).n(), (3438799/6994159).n(), (879060/6994159).n()

(0.000000000000000,
 0.0543779459403196,
 0.124441837824962,
 0.0879794125355171,
 0.115848667438072,
 0.491667261210390,
 0.125684875050739)

In [11]:
#optim = blowup_construction(CTGp, 6, 3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], symmetric=True, variable=True)
v1 = subs(optim, [1/3, 1/3, 1/3]).values()
v2 = subs(optim, [1/3 + 1/100, 1/3 - 1/200, 1/3 - 1/200]).values()
print(print_sparse(v1.n()))
print("\n\n")
print(print_sparse(v2.n()))

0: 0.0041
1: 0.0494
2: 0.1235
4: 0.0823
463: 0.1235
2476: 0.4938
2839: 0.1235



0: 0.0041
1: 0.0496
2: 0.1236
4: 0.0824
463: 0.1235
2476: 0.4935
2839: 0.1233


In [5]:
loglevel = 1

def set_sum(self):
    valvec = self.values()
    if len(valvec)==0:
        return self
    par = valvec[0].parent()
    gs = par.gens()
    repl = {gs[-1]: 1 - sum(gs[:-1])}
    nvec = vector([xx.subs(repl) for xx in valvec])
    return self.parent()(self.size(), nvec)

def subs(self, args):
    valvec = self.values()
    if len(valvec)==0:
        return self
    par = valvec[0].parent()
    gs = par.gens()
    repl = {gs[ii]:args[ii] for ii in range(min(len(args), len(gs)))}
    nvec = vector([xx.subs(repl) for xx in valvec])
    retalg = FlagAlgebra(QQ, self.parent().theory())
    return retalg(self.size(), nvec)

def derivative(self, times):
    valvec = self.values()
    if len(valvec)==0:
        return self
    par = valvec[0].parent()
    gs = par.gens()
    rvec = []
    for ff, vv in enumerate(valvec):
        if vv==0:
            rvec.append(0)
            continue
        aval = vv
        for ii in range(min(len(times), len(gs))):
            aval = aval.derivative(gs[ii], times[ii])
        rvec.append(aval)
    return self.parent()(self.size(), vector(rvec))

def derivatives(self, point):
    times = []
    for xx in self.values():
        if xx==0:
            continue
        dd = xx.dict()
        for kk in dd.keys():
            kk = tuple(kk)
            if kk in times:
                continue
            for ll in itertools.product(*[range(ii+1) for ii in kk]):
                if ll not in times:
                    times.append(ll)
    res = []
    if loglevel>2:
        print("derivatives:")
        print(times)
    for xx in times:
        der = subs(derivative(self, xx), point)
        minnz = 1000000
        for xx in der.values():
            if int(xx)!=0:
                minnz = min(abs(xx), minnz)
        if minnz != 1000000:
            res.append(der/minnz)
    return res

def blowup_construction(self, target_size, construction_size, variable=False, symmetric=False, unordered=False, **kwargs):
    from tqdm import tqdm
    from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
    R = PolynomialRing(QQ, construction_size, "X")
    gs = R.gens()
    res = 0

    if symmetric:
        terms = ((sum(gs))**target_size).dict()
        if loglevel>0:
            iterator = tqdm(terms)
        else:
            itertor = terms
        for exps in iterator:
            verts = []
            for ind, exp in enumerate(exps):
                verts += [ind]*exp
            coeff = terms[exps]/(construction_size**target_size)
            if variable:
                coeff = terms[exps]
                for ind, exp in enumerate(exps):
                    coeff *= gs[ind]**exp
            blocks = {}
            for rel in kwargs:
                if rel not in self.signature():
                    continue
                reledges = kwargs[rel]
                bladd = []
                for edge in reledges:
                    clusters = [[ii for ii in range(target_size) if verts[ii]==ee] for ee in edge]
                    bladd += list(set([tuple(sorted(xx)) for xx in itertools.product(*clusters) if len(set(xx))==len(edge)]))
                blocks[rel] = bladd
            res += self(target_size, **blocks).afae()*coeff
    else:
        rep = int(target_size if not unordered else target_size - 1)
        if loglevel>0:
            iterator = tqdm(itertools.product(range(construction_size), repeat=rep))
        else:
            iterator = itertools.product(range(construction_size), repeat=rep)
        for verts in iterator:
            if unordered:
                verts = [0] + list(verts)

            coeff = 1/(construction_size**rep)
            if variable:
                coeff = 1
                for ind in verts:
                    coeff *= gs[ind]
            
            blocks = {}
            for rel in kwargs:
                if rel not in self.signature():
                    continue
                reledges = kwargs[rel]
                bladd = []
                for edge in reledges:
                    clusters = [[ii for ii in range(target_size) if verts[ii]==ee] for ee in edge]
                    bladd += list(set([tuple(sorted(xx)) for xx in itertools.product(*clusters) if len(set(xx))==len(edge)]))
                blocks[rel] = bladd
            res += self(target_size, **blocks).afae() * coeff
    return res

def sym_asym_bases(self, n, ftype=None):
    r"""
    Generate the change of base matrices for the symmetric
    and the asymmetric subspaces
    """
    
    flags = self.generate_flags(n, ftype)
    uniques = []
    sym_base = []
    asym_base = []
    for xx in flags:
        xxid = self.identify(n, [xx.ftype()], **xx.blocks())
        if xxid not in uniques:
            uniques.append(xxid)
            sym_base.append(xx.afae())
        else:
            sym_ind = uniques.index(xxid)
            asym_base.append(sym_base[sym_ind] - xx.afae())
            sym_base[sym_ind] += xx
    m_sym = matrix(len(sym_base), len(flags), [xx.values() for xx in sym_base], sparse=True)
    m_asym = matrix(len(asym_base), len(flags), [xx.values() for xx in asym_base], sparse=True)
    return m_sym, m_asym

#for quick sparse matrix printing
def print_sparse(ls, eps=1e-6):
    if loglevel<=0:
        return
    nzs = [(ii, ls[ii]) for ii in range(len(ls)) if abs(ls[ii])>eps]
    if isinstance(ls[0], Rational):
        st = "\n".join(["{}: {}".format(nn[0], nn[1]) for nn in nzs])
    else:
        st = "\n".join(["{}: {:.4f}".format(nn[0], float(nn[1])) for nn in nzs])
    return st

#for debug printing values with their variable names
def print_debug(*args):
    if loglevel<=1:
        return
    import inspect
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = s.split('(')[1].split(')')[0].split(',')
    names = [name.strip() for name in r]
    
    for name, value in zip(names, args):
        if isinstance(value, str):
            print(value)
        else:
            print(f"{name}: {value}")

#continued fraction rounding
def cfr(value, quotient_bound=7, denom_bound=9):
    cf = continued_fraction(value)
    for ii, xx in enumerate(cf.quotients()):
        if xx>=2**quotient_bound or cf.denominator(ii)>2**(denom_bound):
            if ii>0:
                return cf.convergent(ii-1)
            return 0
    return cf.value()

#rounding for lists
def cfr_list(ls, force_pos=False):
    if force_pos:
        return [max(cfr(xx), 0) for xx in ls]
    else:
        return [cfr(xx) for xx in ls]

#rounding for matrices
def cfr_matrix(mat):
    return matrix(QQ, [cfr_list(xx, False) for xx in mat])

#rounding matrix based on LDL factoring
def cfr_ldl(mat):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, cfr_matrix(mat_ldl[1]))
    D = diagonal_matrix(QQ, cfr_list(mat_ldl[2].diagonal(), True))
    pl = P*L
    return pl*D*pl.T

#rounding based on simple denominator scaling
def sdem(value, den=1024):
    return floor(value*den)/den

#rounding for lists
def sdem_list(ls, force_pos=False):
    if force_pos:
        return [max(sdem(xx), 0) for xx in ls]
    else:
        return [sdem(xx) for xx in ls]

#rounding for matrices
def sdem_matrix(mat):
    return matrix(QQ, [sdem_list(xx, False) for xx in mat])

#rounding matrix based on LDL factoring
def sdem_ldl(mat):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, sdem_matrix(mat_ldl[1]))
    D = diagonal_matrix(QQ, sdem_list(mat_ldl[2].diagonal(), True))
    pl = P*L
    return pl*D*pl.T

#adaptive continued fraction rounding, seems to work well
def cfr_adaptive(ls, onevec):
    best_vec = None
    best_error = 1000
    best_lcm = 1000000000
    
    orig = vector(ls)
    for resol1 in range(5, 20):
        resol2 = round(resol1*1.5)
        rls = vector([cfr(xx, resol1, resol2) for xx in ls])
        ip = rls*onevec
        if ip != 0 and abs(ip - 1)<best_error:
            if ip.as_integer_ratio()[1] > best_lcm**1.5 and ip != 1:
                continue
            best_vec = rls/ip
            best_error = abs(ip - 1)
            best_lcm = ip.as_integer_ratio()[1]
    return best_vec


#helper function, to add the kernel constraints to the table constructor based on a provided phi vector
def adjust_table_phi(self, table_constructor, phi_vectors_exact, test=False):
    
    for param in table_constructor.keys():
        ns, ftype = param
        table = self.mul_project_table(ns, ns, ftype, [])
        Zs = [[None for _ in range(len(phi_vectors_exact))] for _ in range(len(table_constructor[param]))]
        for gg, morig in enumerate(table):
            for ii, base in enumerate(table_constructor[param]):
                mat = base * morig * base.T
                for phind, phi_vector_exact in enumerate(phi_vectors_exact):
                    if Zs[ii][phind]==None:
                        Zs[ii][phind] = mat*phi_vector_exact[gg]
                    else:
                        Zs[ii][phind] += mat*phi_vector_exact[gg]
        
        new_bases = []
        for ii, Zgroup in enumerate(Zs):
            Z = None
            for Zjj in Zgroup:
                if test and min(Zjj.eigenvalues())<0 and loglevel>0:
                    print("Construction based Z matrix for {} is not semidef: {}".format(ftype, min(Zjj.eigenvalues())))
                if Z==None:
                    Z = Zjj
                else:
                    Z.augment(Zjj)
            Zk = Z.kernel()
            print_debug("\n\n", Z, Zk)
            Zkern = Zk.basis_matrix()
            if Zkern.nrows()>0:
                new_bases.append(Zkern * table_constructor[param][ii])
        table_constructor[param] = new_bases

    return table_constructor

def print_eigenvalues(self, table_constructor, sdp_result):
    block_index = 0
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        
        for plus_index, base in enumerate(table_constructor[params]):
            X_approx = matrix(sdp_result['X'][block_index + plus_index])
            X_eigenvalues = X_approx.eigenvalues()
            print_debug(ftype, plus_index, X_eigenvalues, "\n\n")
            
        block_index += len(table_constructor[params])

#helper function, moves a table to an sdp input data
def tables_to_sdp_data(self, table_constructor, prev_data=None):
    if prev_data==None:
        mat_inds = []
        mat_vals = []
        block_sizes = []
    else:
        mat_inds, mat_vals, block_sizes = prev_data
    block_index = len(block_sizes) + 1
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        block_sizes += [base.nrows() for base in table_constructor[params]]
        
        #only loop through the table once
        for gg, morig in enumerate(table):
            #for each base change create the entries
            for plus_index, base in enumerate(table_constructor[params]):
                mm = base * morig * base.T
                dd = mm._dict()
                if len(dd)>0:
                    inds, values = zip(*mm._dict().items())
                    iinds, jinds = zip(*inds)
                    for cc in range(len(iinds)):
                        if iinds[cc]>=jinds[cc]:
                            mat_inds.extend([gg+1, block_index + plus_index, iinds[cc]+1, jinds[cc]+1])
                            mat_vals.append(values[cc])
        block_index += len(table_constructor[params])
    return mat_inds, mat_vals, block_sizes

#helper function, moves the linear constraints to the sdp input data
def constraints_to_sdp_data(self, flag_num, constraints_vals, constraints_flags_vec, prev_data=None):
    if prev_data==None:
        mat_inds = []
        mat_vals = []
        block_sizes = []
    else:
        mat_inds, mat_vals, block_sizes = prev_data
    block_index = len(block_sizes) + 1

    constr_num = len(constraints_vals)
    
    for ii in range(constr_num):
        mat_inds.extend([0, block_index+1, 1+ii, 1+ii])
        mat_vals.append(constraints_vals[ii])
    
    for gg in range(flag_num):
        mat_inds.extend([gg+1, block_index, gg+1, gg+1])
        mat_vals.append(1)
        for ii in range(constr_num):
            mat_inds.extend([gg+1, block_index+1, ii+1, ii+1])
            mat_vals.append(constraints_flags_vec[ii][gg])
    block_sizes += [-flag_num, -constr_num]
    
    return mat_inds, mat_vals, block_sizes

#flatten and unflatten symmetric matrices
def flatten_matrix(mat, doubled=False):
    res = []
    factor = 2 if doubled else 1
    for ii in range(len(mat)):
        res.append(mat[ii][ii])
        res += [factor*mat[ii][jj] for jj in range(ii+1, len(mat))]
    return res

def unflatten_matrix(ls, dim, doubled=False):
    mat = [[0]*dim for ii in range(dim)]
    factor = 2 if doubled else 1
    index = 0
    for ii in range(dim):
        # Fill the diagonal element
        mat[ii][ii] = ls[index]
        index += 1
        # Fill the off-diagonal elements
        for jj in range(ii + 1, dim):
            mat[ii][jj] = ls[index] / factor
            mat[jj][ii] = ls[index] / factor
            index += 1
    return matrix(mat), ls[index:]

#round the sdp output based on slacks
def round_sdp_solution(self, sdp_result, table_constructor, block_sizes, target_vector_exact, phi_vectors_exact, positives_matrix_exact):
    
    if loglevel>=1:
        print("Begin rounding\n")

    phi_vector_exact = phi_vectors_exact[0]
    
    positives_matrix_exact = positives_matrix_exact[:-2, :] # remove the equality constraints
    print_debug(block_sizes, target_vector_exact, phi_vector_exact, positives_matrix_exact)
    
    flags_num = -block_sizes[-2] # same as |F_n|
    
    c_vector_approx = vector(sdp_result['X'][-2]) # dim: |F_n|, c vector, primal slack for flags
    c_vector_rounded = vector(sdem_list(c_vector_approx)) # as above but rounded
    
    # The F (FF) flag indecies where the c vector is zero/nonzero
    c_zero_inds = [FF for FF, xx in enumerate(c_vector_approx) if (abs(xx)<1e-6 or phi_vector_exact[FF]!=0)]
    c_nonzero_inds = [FF for FF in range(flags_num) if FF not in c_zero_inds]    

    if loglevel>=2:
        print("\nc values and related\n")
    print_debug(flags_num, c_vector_approx, c_vector_rounded, c_zero_inds, c_nonzero_inds)
    
    
    
    positives_num = -block_sizes[-1] - 2 # same as m, number of positive constraints (-2 for the equality)
    
    phi_pos_vector_exact = positives_matrix_exact*phi_vector_exact # dim: m, witness that phi is positive
    
    e_vector_approx = vector(sdp_result['X'][-1][:-2]) # dim: m, the e vector, primal slack for positivitives
    e_vector_rounded = vector(sdem_list(e_vector_approx)) # as above but rounded
    
    # The f (ff) positivity constraints where the e vector is zero/nonzero
    e_zero_inds = [ff for ff, xx in enumerate(e_vector_approx) if (abs(xx)<1e-6 or phi_pos_vector_exact[ff]!=0)]
    e_nonzero_inds = [ff for ff in range(positives_num) if ff not in e_zero_inds]

    if loglevel>=2:
        print("\ne values and related\n")
    print_debug(positives_num, phi_pos_vector_exact, e_vector_approx, e_vector_rounded, e_zero_inds, e_nonzero_inds)
    
    
    
    bound_exact = target_vector_exact*phi_vector_exact # the u value, the bound we want to prove
    # the constraints for the flags that are exact
    corrected_target_relevant_exact = vector([target_vector_exact[FF] - bound_exact for FF in c_zero_inds])
    # the d^f_F matrix, but only the relevant parts for the rounding
    # so F where c_F = 0 and f where e_f != 0
    positives_matrix_relevant_exact = matrix(QQ, len(e_nonzero_inds), len(c_zero_inds), [[positives_matrix_exact[ff][FF] for FF in c_zero_inds] for ff in e_nonzero_inds])
    # the e vector, but only the nonzero entries
    e_nonzero_list_rounded = [e_vector_rounded[ff] for ff in e_nonzero_inds]

    if loglevel>=2:
        print("\nrelevant parameters for stuff\n")
    print_debug(bound_exact, corrected_target_relevant_exact, positives_matrix_relevant_exact, e_nonzero_list_rounded)
    
    # 
    # Flatten the matrices relevant for the rounding
    # 
    # M table transforms to a matrix, (with nondiagonal entries doubled)
    # only the FF index matrices corresponding with tight constraints are used
    # 
    # X transforms to a vector
    # only the semidefinite blocks are used
    # 
    
    # The relevant entries of M flattened to a matrix this will be indexed by 
    # c_zero_inds and the triples from the types
    M_flat_relevant_matrix_exact = matrix(QQ, len(c_zero_inds), 0, 0, sparse=True)
    X_flat_vector_rounded = [] # The rounded X values flattened to a list
    block_index = 0
    block_info = []
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        
        for plus_index, base in enumerate(table_constructor[params]):
            block_info.append([ftype, base])
            X_approx = sdp_result['X'][block_index + plus_index]
            X_flat_vector_rounded += sdem_list(flatten_matrix(X_approx))
            
            M_extra = []
            
            for FF in c_zero_inds:
                M_FF = table[FF]
                M_extra.append(flatten_matrix((base * M_FF * base.T).rows(), doubled=True))
            
            M_flat_relevant_matrix_exact = M_flat_relevant_matrix_exact.augment(matrix(M_extra))
        block_index += len(table_constructor[params])
    
    # 
    # Append the relevant M matrix and the X with the additional values from
    # the positivity constraints. 
    #
    # Then correct the x vector values
    # 
    
    M_matrix_final = M_flat_relevant_matrix_exact.augment(positives_matrix_relevant_exact.T)
    x_vector_final = vector(X_flat_vector_rounded+e_nonzero_list_rounded)

    # correct the values of the x vector, based on the minimal L_2 norm
    x_vector_corr = x_vector_final - M_matrix_final.T * \
    (M_matrix_final * M_matrix_final.T).pseudoinverse() * \
    (M_matrix_final*x_vector_final - corrected_target_relevant_exact)

    if loglevel>=2:
        print("\nrelevant M and X stuff flattened and correction\n")
    dim_M_flat = M_flat_relevant_matrix_exact.dimensions()
    dim_pos = positives_matrix_relevant_exact.dimensions()
    print_debug(dim_M_flat, dim_pos, M_flat_relevant_matrix_exact, X_flat_vector_rounded, M_matrix_final, x_vector_final, x_vector_corr)
    mtm = (M_matrix_final * M_matrix_final.T)
    print_debug(mtm)
    
    #
    # Recover the X matrices and e vector from the corrected x
    #
    
    e_nonzero_vector_corr = x_vector_corr[-len(e_nonzero_inds):]
    e_vector_corr = vector(QQ, positives_num, dict(zip(e_nonzero_inds, e_nonzero_vector_corr)))
    
    if min(e_vector_corr)<0 and loglevel>=1:
        print("Linear coefficient is negative: {}".format(min(e_vector_corr)))
    
    X_matrix_corr = []
    for ii, block_dim in enumerate(block_sizes):
        if block_dim<0:
            break
        X_matrix_ii_corr, x_vector_corr = unflatten_matrix(x_vector_corr, block_dim)
        X_matrix_corr.append(matrix(X_matrix_ii_corr))
        if min(X_matrix_ii_corr.eigenvalues())<0 and loglevel>=1:
            print("Rounded X matrix is not semidefinite: {}".format(min(X_matrix_ii_corr.eigenvalues())))
    X_matrix_corr.append(e_vector_corr)
    
    
    #
    # Verify the bound and semidefiniteness
    #

    block_index = 0
    slacks = target_vector_exact - positives_matrix_exact.T*e_vector_corr
    
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        
        for plus_index, base in enumerate(table_constructor[params]):
            X_flat_vector_corr = vector(flatten_matrix(X_matrix_corr[block_index + plus_index].rows()))
            
            for gg, morig in enumerate(table):
                mm = base * morig * base.T
                M_flat_vector_exact = vector(flatten_matrix((base * morig * base.T).rows(), doubled=True))
                slacks[gg] -= M_flat_vector_exact*X_flat_vector_corr
        
        block_index += len(table_constructor[params])

    if loglevel>=2:
        print("\nfinal_values\n")
    print_debug(e_nonzero_vector_corr, e_vector_corr, positives_matrix_exact.T*e_vector_corr, slacks, X_matrix_corr)
    
    return min(slacks), X_matrix_corr

#the combined code
def optimize_problem(self, target_element, target_size, maximize=True, positives=None, construction=None, certificate=False, rounding=True):
    from csdpy import solve_sdp
    from tqdm import tqdm
    import sys
    import io
    import time
    
    #
    # initial setup
    #
    
    base_flags = self.generate_flags(target_size)
    if loglevel>=1:
        print("base flags generated")
    mult = -1 if maximize else 1
    target_vector_exact = (target_element.project()*(mult)<<(target_size - target_element.size())).values()
    if loglevel>=1:
        print("avals constructed")
    
    #
    # create the table data
    #
    
    ftype_flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
              for flag in self.generate_flags(kk)]
    ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
              for flag in ftype_flags]
    if loglevel>=1:
        print("ftypes constructed")
    flags = [self.generate_flags((target_size + ftype.size())//2, ftype) for ftype in ftypes]
    flag_sizes = [len(xx) for xx in flags]
    if loglevel>=1:
        print("typed flags constructed, their length is {}".format(flag_sizes))
    
    sym_asym_mats = [sym_asym_bases(self, (target_size + ftype.size())//2, ftype) for ftype in ftypes]
    if loglevel>=1:
        print("sym and asym matrices constructed")
    
    table_constructor = {}
    for ii, ftype in (pbar := tqdm(enumerate(ftypes))):
        ns = (target_size + ftype.size())//2
        #pre-calculate the table here
        table = self.mul_project_table(ns, ns, ftype, [])
        sym_base, asym_base = sym_asym_mats[ii]
        bases = []
        if sym_base.nrows()!=0:
            bases.append(sym_base)
        if asym_base.nrows()!=0:
            bases.append(asym_base)
        table_constructor[(ns, ftype)] = bases
        pbar.set_description("done with mult table for {}".format(ftype))
    if loglevel>=1:
        print("tables constructed")
    
    sdp_data = tables_to_sdp_data(self, table_constructor)
    if loglevel>=1:
        print("tables added to sdp data")
    
    #
    # add constraints data
    #
    
    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
            if loglevel>=1:
                print("done with constraint for \n{}\n".format(fv))
        constraints_vals = [0]*len(constraints_flags)
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])
    positives_list_exact = [(xx<<(target_size-xx.size())).values() for xx in constraints_flags]
    positives_matrix_exact = matrix(positives_list_exact)
    if loglevel>=1:
        print("constraints constructed")
    
    sdp_data = constraints_to_sdp_data(self, len(base_flags), constraints_vals, positives_list_exact, sdp_data)
    if loglevel>=1:
        print("constraints added to sdp data")
    
    #
    # if no y value provided, run the optimizer first, only to get the y values
    #
    if (not rounding):
        if loglevel>=1:
            print("running sdp without construction")
        
        mat_inds, mat_vals, block_sizes = sdp_data
    
        time.sleep(float(0.1))
        initial_sol = solve_sdp(block_sizes, list(target_vector_exact), mat_inds, mat_vals)
        time.sleep(float(0.1))
        
        phi_vector_original = initial_sol['y']
        phi_vector_rounded = cfr_adaptive(initial_sol['y'], one_vector.values())
        if loglevel>=1:
            print("rounded phi vector is: \n{}".format(print_sparse(phi_vector_rounded)))
        return initial_sol['primal']*(-1 if maximize else 1), phi_vector_rounded

    if isinstance(construction, Flag):
        phi_vectors_exact = [construction.values()]
    else:
        phi_vectors_exact = [xx.values() for xx in construction]
    
    #
    # adjust the table to consider the kernel from y_rounded
    #
    
    table_constructor = adjust_table_phi(self, table_constructor, phi_vectors_exact)
    sdp_data = tables_to_sdp_data(self, table_constructor)
    sdp_data = constraints_to_sdp_data(self, len(base_flags), constraints_vals, positives_list_exact, sdp_data)
    mat_inds, mat_vals, block_sizes = sdp_data
    
    if loglevel>=1:
        print("running SDP after kernel correction with block sizes {}".format(block_sizes))
    
    time.sleep(float(0.1))
    final_sdp = solve_sdp(block_sizes, list(target_vector_exact), mat_inds, mat_vals)
    time.sleep(float(0.1))
    
    res = round_sdp_solution(self, final_sdp, table_constructor, block_sizes, target_vector_exact, phi_vectors_exact, positives_matrix_exact)

    for ii, Xii in enumerate(res[1]):
        if block_sizes[ii]<=0:
            continue
        X_eigenvalues = matrix(Xii).eigenvalues()
        print_debug(X_eigenvalues, "\n\n")
    
    if maximize:
        res = (-res[0], res[1])
    if certificate:
        return res
    else:
        return res[0]